In [ ]:
import cv2  # OpenCV Library
cv2.__version__

In [ ]:
#-----------------------------------------------------------------------------
#       Load and configure Haar Cascade Classifiers
#-----------------------------------------------------------------------------
 
# location of OpenCV Haar Cascade Classifiers:
#baseCascadePath = "OpenCV/haarcascades/"
 
# xml files describing our haar cascade classifiers
faceCascadeFilePath = "./haarcascade_frontalface_default.xml"
noseCascadeFilePath = "./haarcascade_mcs_nose.xml" ## need to download this and put it in local (same with face)
 
# build our cv2 Cascade Classifiers
faceCascade = cv2.CascadeClassifier(faceCascadeFilePath)
noseCascade = cv2.CascadeClassifier(noseCascadeFilePath)

## RenderableImage Base Class

This is a generic renderable image. It is able to be loaded and has a render method. The individual child classes have to implement their own methods for updating their location since the position of a hat and a position of a mustache are fundamentally different.

In [ ]:
class RenderableImage:
    def __init__(self, img_path):
        img = cv2.imread(img_path, -1)
        self.bgr_img = img[:, :, 0:3]       
        self.mask = img[:, :, 3]
        self.mask_inv = cv2.bitwise_not(self.mask)
        
        original_height, original_width = img.shape[:2]
        self.original_height = original_height
        self.original_width = original_width
        print(self)
    
    def render(self, width, height, roi_color):
        image = self.bgr_img
        mask = self.mask
        mask_inv = self.mask_inv
        
        image_resize = cv2.resize(image, (width,height), interpolation = cv2.INTER_AREA)
        mask_resize = cv2.resize(mask, (width,height), interpolation = cv2.INTER_AREA)
        mask_inv_resize = cv2.resize(mask_inv, (width,height), interpolation = cv2.INTER_AREA)
        
        x1, x2, y1, y2 = self.dimensions
        if y1 > 0:
            roi = roi_color[y1:y2, x1:x2]

            roi_bg = cv2.bitwise_and(roi,roi,mask = mask_inv_resize)
            roi_fg = cv2.bitwise_and(image_resize,image_resize, mask = mask_resize)

            dst = cv2.add(roi_bg,roi_fg)
            roi_color[y1:y2, x1:x2] = dst

## HatImage Class

In [ ]:
class HatImage(RenderableImage):
    def calculate_dimensions(self, x, y, w, h):
        x1 = 0
        x2 = int(w)
        y1 = 0
        y2 = int(h/2)

        hat_width = x2 - x1
        hat_height = y2 - y1
        self.dimensions = [x, x+w, y-hat_height, y]
        return hat_width, hat_height

## MustacheImage Class

In [ ]:
class MustacheImage(RenderableImage):
    def calculate_dimensions(self, x, y, w, h, clip_width, clip_height):
        mustacheWidth =  3 * w
        mustacheHeight = mustacheWidth * self.original_height / self.original_width

        # Center the mustache on the bottom of the nose
        x1 = int(x - (mustacheWidth/4))
        x2 = int(x + w + (mustacheWidth/4))
        y1 = int(y + h - (mustacheHeight/2))
        y2 = int(y + h + (mustacheHeight/2))

        # Check for clipping
        if x1 < 0:
            x1 = 0
        if y1 < 0:
            y1 = 0
        if x2 > clip_width:
            x2 = clip_width
        if y2 > clip_height:
            y2 = clip_height

        # Re-calculate the width and height of the mustache image
        mustacheWidth = x2 - x1 
        mustacheHeight = y2 - y1
        self.dimensions = (x1, x2, y1, y2)
        return mustacheWidth, mustacheHeight

In [ ]:
mustache = MustacheImage('./mustache.png')
hat = HatImage('./harvard_hat.png')

In [ ]:
video_capture = cv2.VideoCapture(0)
 
while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = 0
        #flags=cv2.CASCADE_SCALE_IMAGE
        #flags=cv2.cv.CV_HAAR_SCALE_IMAGE
    )
    for (x, y, w, h) in faces:
        # Un-comment the next line for debug (draw box around all faces)
        face = cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        # Detect a nose within the region bounded by each face (the ROI)
        nose = noseCascade.detectMultiScale(roi_gray)
        for (nx,ny,nw,nh) in nose:
            # Un-comment the next line for debug (draw box around the nose)
            #cv2.rectangle(roi_color,(nx,ny),(nx+nw,ny+nh),(255,0,0),2)
            
            width, height = mustache.calculate_dimensions(
                nx, ny, nw, nh, clip_width=w, clip_height=h
            )
            mustache.render(width, height, roi_color)
            
            hat_width, hat_height = hat.calculate_dimensions(x, y, w, h)
            hat.render(hat_width, hat_height, frame)
            break
 
    # Display the resulting frame
    cv2.imshow('Video', frame)
 
    # press any key to exit
    # NOTE;  x86 systems may need to remove: "0xFF == ord('q')"
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
 
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [ ]:
# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()